In [ ]:
'''
实现中的问题：
１．　adaboost的思路很清晰，知道alpha和ｗ的计算公式后，整个算法很好描述
难点反而在弱分类器那里，sklearn里一般用ＣＡＲＴ决策树，我们这里用的是树桩，即只有三个节点的二叉树，只切分一次
按理说，应该可以直接拿已经实现的决策树作为这里的弱分类器，这里没有尝试。

２．　坑都在弱分类器的实现里，选择最优特征的最优二值点，记录error,yp等。
这里有一个learning rate，用来作为切分特征值的步长

３．　原代码，在弱分类器中设置了停止寻找特征的条件，即error＝＝０
这里，没有做这样的设置，而是在每次的弱分类器确定后，统一计算了当前分类器的error，如果==0,则提前结束，不必等执行完所有的n_estimator.
'''


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math


In [7]:
#load data与感知机所用相同
data=load_iris()
# print(data)
df=pd.DataFrame(data.data,columns=data.feature_names)
df['label']=data.target
df.columns=['sl','sw','pl','pw','label']

# X,y
X=np.array(df.iloc[:100,[0,1]])
y=np.array(df.iloc[:100,-1])

y=np.array([-1 if i==0 else 1 for i in y  ])

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,shuffle=False)


In [10]:
class AdaBoostClassifier():
    def __init__(self,n_estimator=50,lr=0.5):
        self.n_estimator=n_estimator
        self.lr=lr
    
    def _init_params(self,X_train,y_train):
        self.X=X_train
        self.y=y_train
        l=len(self.X)
        self.D=[1/l]*l
        self.clf_alpha=[]
    
    
    def _alpha(self,error):
        return 0.5*math.log((1-error)/error)
    
    def _w_update(self,D:list,yp:list,alpha):
        wp=[]
        Z=sum([D[i]*math.exp(-alpha*self.y[i]*yp[i]) for i in range(len(D))])
        
        for i in range(len(D)):
            wp.append(D[i]*math.exp(-alpha*self.y[i]*yp[i])/Z)
        return wp
    
    def ferror(self):
        fyp=[0.0]*len(self.y)
        for alpha,bc in self.clf_alpha:
            fyp+=alpha*np.array(bc.yp)
        fyp=[1 if i>0 else -1 for i in fyp]
        return sum([0 if fyp[i]==self.y[i] else 1 for i in range(len(fyp))])/len(self.y)
            
    def fit(self,X_train,y_train):
        self._init_params(X_train,y_train)
        for i in range(self.n_estimator):
            bc=BasicClassifier(w=self.D,lr=self.lr)
            bc.fit(X_train,y_train)
            alpha=self._alpha(bc.error)
            self.D=self._w_update(self.D,bc.yp,alpha)
            self.clf_alpha.append((alpha,bc))
            if self.ferror()==0:
                break
    
    def score(self,X_test,y_test):
        yp=[self.predict(xp) for xp in X_test]
        return sum([1 if yp[i]==y_test[i] else 0 for i in range(len(yp))])/len(y_test)
        
    def predict(self,xp):
        yp=0
        for alpha,bc in self.clf_alpha:
            yp+=alpha*bc.predict(xp)
        if yp>0:
            return 1
        else:
            return -1
            
class BasicClassifier():
    def __init__(self,w,lr=0.5):
        self.lr=lr
        self.w=w
        self.fi=None
        self.vi=None
        self.direction=None
        self.error=math.inf
        self.yp=[]
    def fit(self,X_train,y_train):
        X_features=list(zip(*X_train))
        for i in range(len(X_train[0])):
            min_f=min(X_features[i])
            max_f=max(X_features[i])
            n_step=(max_f-min_f+self.lr)//self.lr
            
            for j in range(int(n_step)):
                error=math.inf
                direct=None
                yp_array=[]
                v=min_f+j*self.lr
                pos_array=[1 if f>=v else -1 for f in X_features[i]]
                pos_error=sum([self.w[e] if pos_array[e]!=y_train[e] else 0 for e in range(len(pos_array))])
                neg_array=[-1 if f>=v else 1 for f in X_features[i]]
                neg_error=sum([self.w[e] if neg_array[e]!=y_train[e] else 0 for e in range(len(neg_array))])
                
                if pos_error>neg_error:
                    error=neg_error
                    yp_array=neg_array
                    direct='negtive'
                else:
                    error=pos_error
                    yp_array=pos_array
                    direct='postive'
                    
                if error<self.error:
                    self.fi=i
                    self.yp=yp_array
                    self.error=error
                    self.vi=v
                    self.direction=direct
    def fx(self,x,direct,fi,vi):
        if direct=='postive':
            
            if x[fi]>=vi:
                return 1
            else:
                return -1
        else:
            if x[fi]>=vi:
                return -1
            else:
                return 1
        
    def predict(self,xp):
        return self.fx(xp,self.direction,self.fi,self.vi)


In [11]:
ada=AdaBoostClassifier()
ada.fit(X_train,y_train)
ada.score(X_test,y_test)

0.9

In [15]:
from sklearn.ensemble import AdaBoostClassifier
clf=AdaBoostClassifier(n_estimators=10,learning_rate=0.2,algorithm='SAMME.R')
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.9